In [1]:
import sys
sys.path.append("..")


In [2]:
import pandas as pd
import numpy as np

In [3]:
df_raw = pd.DataFrame([
    {'item_id': 0, 'user_id': 1, 'output': 3},
    {'item_id': 0, 'user_id': 2, 'output': 3},
    {'item_id': 1, 'user_id': 1, 'output': 3},
    {'item_id': 1, 'user_id': 2, 'output': 3},
    {'item_id': 2, 'user_id': 1, 'output': 3},
    {'item_id': 2, 'user_id': 2, 'output': 3},
    {'item_id': 3, 'user_id': 1, 'output': 2},
    {'item_id': 3, 'user_id': 2, 'output': 3},
    {'item_id': 4, 'user_id': 1, 'output': 3},
    {'item_id': 4, 'user_id': 2, 'output': 3},
    {'item_id': 5, 'user_id': 1, 'output': 3},
    {'item_id': 5, 'user_id': 2, 'output': 3},
    {'item_id': 6, 'user_id': 1, 'output': 3},
    {'item_id': 6, 'user_id': 2, 'output': 2},
    {'item_id': 7, 'user_id': 1, 'output': 3},
    {'item_id': 7, 'user_id': 2, 'output': 3},
    {'item_id': 8, 'user_id': 1, 'output': 3},
    {'item_id': 8, 'user_id': 2, 'output': 3},
    {'item_id': 9, 'user_id': 1, 'output': 3},
    {'item_id': 9, 'user_id': 2, 'output': 3},
    {'item_id': 10, 'user_id': 1, 'output': 3},
    {'item_id': 10, 'user_id': 2, 'output': 3},
    {'item_id': 11, 'user_id': 1, 'output': 2},
    {'item_id': 11, 'user_id': 2, 'output': 3},
    {'item_id': 12, 'user_id': 1, 'output': 3},
    {'item_id': 12, 'user_id': 2, 'output': 3},
    {'item_id': 13, 'user_id': 1, 'output': 3},
    {'item_id': 13, 'user_id': 2, 'output': 3},
    {'item_id': 14, 'user_id': 1, 'output': 3},
    {'item_id': 14, 'user_id': 2, 'output': 3},
    {'item_id': 15, 'user_id': 1, 'output': 3},
    {'item_id': 15, 'user_id': 2, 'output': 3},
    {'item_id': 16, 'user_id': 1, 'output': 3},
    {'item_id': 16, 'user_id': 2, 'output': 3},
    {'item_id': 17, 'user_id': 1, 'output': 3},
    {'item_id': 17, 'user_id': 2, 'output': 3},
    {'item_id': 18, 'user_id': 1, 'output': 2},
    {'item_id': 18, 'user_id': 2, 'output': 3},
    {'item_id': 19, 'user_id': 1, 'output': 2},
    {'item_id': 19, 'user_id': 2, 'output': 3},
    {'item_id': 20, 'user_id': 1, 'output': 3},
    {'item_id': 20, 'user_id': 2, 'output': 3},
    {'item_id': 21, 'user_id': 1, 'output': 3},
    {'item_id': 21, 'user_id': 2, 'output': 3},
    {'item_id': 22, 'user_id': 1, 'output': 3},
    {'item_id': 22, 'user_id': 2, 'output': 3},
    {'item_id': 23, 'user_id': 1, 'output': 2},
    {'item_id': 23, 'user_id': 2, 'output': 3},
    {'item_id': 24, 'user_id': 1, 'output': 3},
    {'item_id': 24, 'user_id': 2, 'output': 3},
    {'item_id': 25, 'user_id': 1, 'output': 3},
    {'item_id': 25, 'user_id': 2, 'output': 3},
    {'item_id': 26, 'user_id': 1, 'output': 3},
    {'item_id': 26, 'user_id': 2, 'output': 3},
    {'item_id': 27, 'user_id': 1, 'output': 2},
    {'item_id': 27, 'user_id': 2, 'output': 3},
    {'item_id': 28, 'user_id': 1, 'output': 3},
    {'item_id': 28, 'user_id': 2, 'output': 3},
    {'item_id': 29, 'user_id': 1, 'output': 3},
    {'item_id': 29, 'user_id': 2, 'output': 3},
    {'item_id': 30, 'user_id': 1, 'output': 3},
    {'item_id': 30, 'user_id': 2, 'output': 3},
    {'item_id': 31, 'user_id': 1, 'output': 3},
    {'item_id': 31, 'user_id': 2, 'output': 3},
    {'item_id': 32, 'user_id': 1, 'output': 3},
    {'item_id': 32, 'user_id': 2, 'output': 3},
    {'item_id': 33, 'user_id': 1, 'output': 3},
    {'item_id': 33, 'user_id': 2, 'output': 3},
    {'item_id': 34, 'user_id': 1, 'output': 3},
    {'item_id': 34, 'user_id': 2, 'output': 3},
    {'item_id': 35, 'user_id': 1, 'output': 3},
    {'item_id': 35, 'user_id': 2, 'output': 3},
    {'item_id': 36, 'user_id': 1, 'output': 3},
    {'item_id': 36, 'user_id': 2, 'output': 3},
    {'item_id': 37, 'user_id': 1, 'output': 3},
    {'item_id': 37, 'user_id': 2, 'output': 3},
    {'item_id': 38, 'user_id': 1, 'output': 3},
    {'item_id': 38, 'user_id': 2, 'output': 3},
    {'item_id': 39, 'user_id': 1, 'output': 2},
    {'item_id': 39, 'user_id': 2, 'output': 3},
    {'item_id': 40, 'user_id': 1, 'output': 3},
    {'item_id': 40, 'user_id': 2, 'output': 3},
    {'item_id': 41, 'user_id': 1, 'output': 2},
    {'item_id': 41, 'user_id': 2, 'output': 3},
    {'item_id': 42, 'user_id': 1, 'output': 3},
    {'item_id': 42, 'user_id': 2, 'output': 3},
    {'item_id': 43, 'user_id': 1, 'output': 2},
    {'item_id': 43, 'user_id': 2, 'output': 3},
    {'item_id': 44, 'user_id': 1, 'output': 3},
    {'item_id': 44, 'user_id': 2, 'output': 3},
    {'item_id': 45, 'user_id': 1, 'output': 2},
    {'item_id': 45, 'user_id': 2, 'output': 3},
    {'item_id': 46, 'user_id': 1, 'output': 3},
    {'item_id': 46, 'user_id': 2, 'output': 3},
    {'item_id': 47, 'user_id': 1, 'output': 2},
    {'item_id': 47, 'user_id': 2, 'output': 3}
])

In [4]:
df_raw = pd.read_csv('data.csv')

_df = df_raw[["item_id", "output"]]
df = _df.pivot_table(index='item_id', columns="output", aggfunc=len, fill_value=0)

_df = df_raw[["user_id", "output"]]
dfa = _df.pivot_table(index='user_id', columns="output", aggfunc=len, fill_value=0)

dfa[1] = 0
df[1] = 0

df = df[[1, 2, 3]]
df.columns = [0, 1, 2]

## Weighting scheme
Based on https://github.com/jmgirard/mReliability/wiki/Weighting-scheme

In [5]:
def identity_kernel(x, y, q):
    return float(x == y)

def linear_kernel(x, y, q):
    return 1 - abs(x - y) / (q - 1)

def quadratic_kernel(x, y, q):
    return 1 - (x - y)**2 / (q - 1)**2


def get_weights(q, kernel):
    return np.array([
        [
            kernel(x, y, q) for x in range(q)
        ] for y in range(q)
    ])

## Observed agreement
Based on https://github.com/jmgirard/mReliability/wiki/Agreement-or-accuracy

In [6]:
def observed_agreement(df, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)
    
    w = get_weights(q, weights_kernel)
    
    r_star = df.dot(w)
    
    po = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum()/N
    return po

In [7]:
observed_agreement(df)

0.75

## Bennett et al.'s S score
Based on https://github.com/jmgirard/mReliability/wiki/Bennett-et-al.'s-S-score

In [8]:
def s_score(df, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)
    
    w = get_weights(q, weights_kernel)
    
    r_star = df.dot(w)
    
    po = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum()/N
    
    pc = w.sum() / q**2
    
    S = (po - pc) / (1 - pc)
    return S

In [9]:
s_score(df)

0.625

In [10]:
def cohens_kappa(df, dfa, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)

    w = get_weights(q, weights_kernel)

    r_star = df.dot(w)

    po = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum() / N

    p = dfa.div(dfa.sum(axis=1), axis=0)
    r, q = p.shape

    pbar = p.sum(axis=0) / r

    rpbar = r * np.array(pbar).reshape(1, q).T * np.array(pbar).reshape(1, q)
    pg = np.array(p).T.dot(np.array(p))
    s2 = (pg - rpbar) / (r - 1)

    pbarplus = np.array(pbar).reshape(1, q).T * np.array(pbar).reshape(1, q)
    pc = (w * (pbarplus - s2/r)).sum()

    k = (po - pc) / (1 - pc)
    return k

In [11]:
cohens_kappa(df, dfa)

-0.03971119133574028

## Gwet's gamma coefficient
Based on https://github.com/jmgirard/mReliability/wiki/Gwet's-gamma-coefficient

In [12]:
def gwets_gamma(df, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)

    w = get_weights(q, weights_kernel)

    r_star = df.dot(w)

    po = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum() / N

    Tw = w.sum()

    pi = df.div(df.sum(axis=1), axis=0).sum() / N

    pc = (pi*(1 - pi)).values.sum() * Tw / (q*(q-1))

    gamma = (po - pc) / (1 - pc)
    return gamma

In [100]:
gwets_gamma(df)

0.7192982456140351

## Krippendorff's alpha coefficient
Based on https://github.com/jmgirard/mReliability/wiki/Krippendorff's-alpha-coefficient

In [101]:
def krippendorffs_alpha(df, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)

    w = get_weights(q, weights_kernel)

    r_star = df.dot(w)

    po2 = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum()/N
    
    rdash = df.sum(axis=1).sum() / N

    epsilon = 1 / (N*rdash)
    po = po2 * (1-epsilon) + epsilon

    pi = df.div(df.sum(axis=1), axis=0).sum() / N

    pc = (w * np.array(pi).reshape(1, q).T * np.array(pi).reshape(1, q)).sum()

    alpha = (po - pc) / (1 - pc)
    return alpha

In [102]:
krippendorffs_alpha(df)

-0.13095238095238113

## Scott's pi coefficient (Fleiss)
based on https://github.com/jmgirard/mReliability/wiki/Scott's-pi-coefficient

In [103]:
def scotts_pi(df, weights_kernel=identity_kernel):
    N, q = df.shape
    n = df.sum(axis=1)

    w = get_weights(q, weights_kernel)

    r_star = df.dot(w)

    po = ((df * (r_star-1)).sum(axis=1) / (n * (n-1))).sum() / N

    pik = df.div(df.sum(axis=1), axis=0).sum() / N

    pc = (w * np.array(pik).reshape(1, q).T * np.array(pik).reshape(1, q)).sum()

    pi = (po - pc) / (1 - pc)
    return pi

In [104]:
scotts_pi(df)

-0.14285714285714285